<a href="https://colab.research.google.com/github/codefupanda/customer_interaction_summary/blob/master/Build_On_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The basics
import numpy as np
import pandas as pd

import os
import subprocess

**Mount G-Drive to download or save files**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


**Copy Kaggle configuration JSON file to download data from kaggle**

In [3]:
!mkdir /root/.kaggle/

In [4]:
!cp "/content/gdrive/My Drive/Colab Notebooks/.kaggle/kaggle.json" '/root/.kaggle/kaggle.json'

**Clone the code from github**

In [5]:
!git clone https://github.com/codefupanda/customer_interaction_summary.git

Cloning into 'customer_interaction_summary'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 201 (delta 124), reused 114 (delta 48), pack-reused 0
Receiving objects: 100% (201/201), 2.26 MiB | 2.07 MiB/s, done.
Resolving deltas: 100% (124/124), done.


**Init step**: Download required data and dependencies

In [6]:
!cd customer_interaction_summary && make requirements && make data


Aborted!
Makefile:53: recipe for target 'data' failed
make: *** [data] Error 1
^C


**TRAIN**: Train the model, src /models/model_configs.py as the configuration for which models to train

In [21]:
!cd customer_interaction_summary && git pull

Updating 3c72316..3977ece
Fast-forward
 data/processed/isear_test.csv  |  3834 ++---
 data/processed/isear_train.csv | 34494 +++++++++++++++++++--------------------
 2 files changed, 19164 insertions(+), 19164 deletions(-)


In [10]:
!cd customer_interaction_summary && python3 src/data/make_dataset.py data/raw data/processed

2020-09-28 15:07:26,508 - __main__ - INFO - making final data set from raw data
2020-09-28 15:07:26,541 - __main__ - INFO - Removing stopwords
2020-09-28 15:09:00,272 - __main__ - INFO - Lemmatizing strings
2020-09-28 15:10:15,741 - __main__ - INFO - Splitting data
2020-09-28 15:10:15,747 - __main__ - INFO - Saving data
2020-09-28 15:10:15,770 - __main__ - INFO - successfully wrote train and test files


In [22]:
!cd customer_interaction_summary && make train

python3 src/models/train_model.py --input_filepath=data/processed --output_filepath=models/ --pad_sequences_maxlen=1000 --max_words=30000 --epochs=20 --batch_size=128 --output_dim=50
2020-09-28 15:36:03.301778: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-28 15:36:04,815 - __main__ - INFO - starting the training process
2020-09-28 15:36:04,815 - __main__ - INFO - --input_filepath data/processed
2020-09-28 15:36:04,815 - __main__ - INFO - --output_filepath models/
2020-09-28 15:36:04,815 - __main__ - INFO - --pad_sequences_maxlen 1000
2020-09-28 15:36:04,815 - __main__ - INFO - --max_words 30000
2020-09-28 15:36:04,815 - __main__ - INFO - --epochs 20
2020-09-28 15:36:04,815 - __main__ - INFO - --batch_size 128
Found 13614 unique tokens.
Found 400000 word vectors.
Training the model: LSTM
2020-09-28 15:36:11.847972: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic l

**Results are ready**

In [27]:
final_report = pd.read_csv("./customer_interaction_summary/models/final_report.csv")

In [28]:
final_report[final_report['Unnamed: 1'] == 'f1-score']

,Unnamed: 0,Unnamed: 1,1,2,3,4,5,6,7,accuracy,macro avg,weighted avg
2,LSTM,f1-score,0.586667,0.582064,0.376328,0.538462,0.460145,0.353175,0.389791,0.471570,0.469519,0.473580
6,LSTM_Glove,f1-score,0.571429,0.593573,0.377897,0.552585,0.465812,0.367347,0.372514,0.473135,0.471594,0.474625
10,HybridModel,f1-score,0.627151,0.620209,0.413284,0.562814,0.509960,0.391304,0.397059,0.503912,0.503112,0.504658
14,HybridModel_Glove,f1-score,0.549091,0.596745,0.383057,0.526154,0.502836,0.339844,0.402414,0.475222,0.471449,0.478959


In [29]:
final_report[final_report['Unnamed: 1'] == 'recall']

,Unnamed: 0,Unnamed: 1,1,2,3,4,5,6,7,accuracy,macro avg,weighted avg
1,LSTM,recall,0.613546,0.542587,0.322078,0.516779,0.456835,0.386957,0.531646,0.471570,0.481489,0.471570
5,LSTM_Glove,recall,0.510029,0.615686,0.369338,0.540070,0.561856,0.373585,0.367857,0.473135,0.476917,0.473135
9,HybridModel,recall,0.658635,0.593333,0.417910,0.520124,0.561404,0.388489,0.398524,0.503912,0.505488,0.503912
13,HybridModel_Glove,recall,0.547101,0.591398,0.386617,0.454787,0.521569,0.365546,0.446429,0.475222,0.473350,0.475222


In [30]:
final_report[final_report['Unnamed: 1'] == 'precision']

,Unnamed: 0,Unnamed: 1,1,2,3,4,5,6,7,accuracy,macro avg,weighted avg
0,LSTM,precision,0.562044,0.627737,0.452555,0.562044,0.463504,0.324818,0.307692,0.471570,0.471485,0.487202
4,LSTM_Glove,precision,0.649635,0.572993,0.386861,0.565693,0.397810,0.361314,0.377289,0.473135,0.473085,0.482412
8,HybridModel,precision,0.598540,0.649635,0.408759,0.613139,0.467153,0.394161,0.395604,0.503912,0.503856,0.508511
12,HybridModel_Glove,precision,0.551095,0.602190,0.379562,0.624088,0.485401,0.317518,0.366300,0.475222,0.475165,0.489447


In [ ]:
!cp "./customer_interaction_summary/models/final_report.csv" "/content/gdrive/My Drive/Colab Notebooks/.kaggle/"

In [ ]:
!ls "/content/gdrive/My Drive/Colab Notebooks/.kaggle/"

final_report.csv  kaggle.json  siim-isic-melanoma-classification


In [ ]:
from google.colab import files
files.download('./customer_interaction_summary/models/final_report.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>